In [1]:
import re
from collections import defaultdict

from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals
 
from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words
 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer

In [2]:
books = ["data/Rowling, J.K. - HP 1 - Harry Potter and the Sorcerer's Stone.txt",
         "data/Rowling, J.K. - HP 2 - Harry Potter and the Chamber of Secrets.txt",
         "data/Rowling, J.K. - HP 3 - Harry Potter and the Prisoner of Azkaban.txt",
         "data/Rowling, J.K. - HP 4 - Harry Potter and the Goblet of Fire.txt",
         "data/Rowling, J.K. - HP 5 - Harry Potter and the Order of the Phoenix.txt",
         "data/Rowling, J.K. - HP 6 - Harry Potter and the Half-Blood Prince.txt",
         "data/Rowling, J.K. - HP 7 - Harry Potter and the Deathly Hallows.txt"]

In [49]:
pattern = ("(C H A P T E R [A-Z -]+)" +                  # Group 1 selects the chapter number
           "\n+([A-Z \n',.-]+)\\b(?![A-Z]+(?=\.)\\b)" +  # Group 2 selects the chapter title but excludes edgs of all caps word beginning first sentence of the chapter
           "(?![a-z']|[A-Z.])" +                         # chapter title ends with lowercase letters or a period
           "(.*?)" +                                     # Group 3 selects the chapter contents
           "(?=C H A P T E R|This book)")                # chapter contents ends with a new chapter or the end of book
hp = defaultdict(dict)
cnt = 0
for book in books:
    cnt += 1
    title = 'hp' + str(cnt)
    title = book[28:-4]
    with open(book, 'r') as f:
        text = f.read().replace('&rsquo;',"'")
    chapters = re.findall(pattern, text, re.DOTALL)
    chap = 0
    for chapter in chapters:
        chap += 1
        chap_title = chapter[1].replace('\n','')
        chap_text = (chapter[2][3:].replace('&ldquo;', '"')
                                   .replace('&rdquo;', '"')
                                   .replace('&mdash;', '—'))
        chap_text = re.sub('\n*&bull; [0-9]+ &bull; \n*' + chap_title + ' \n*', '', chap_text, flags=re.IGNORECASE)
        chap_text = re.sub('\n*&bull; [0-9]+ &bull; \s*CHAPTER [A-Z]+ \s*', '', chap_text)
        chap_text = re.sub(' \n&bull; [0-9]+ &bull; \n*', '', chap_text)
        chap_text = re.sub('\n+', '\n', chap_text)
        hp[title]['Chapter ' + str(chap)] = (chap_title, chap_text)
hp = dict(hp)

### The format of the Harry Potter hp dictionary is as follows:
#### The first book is 'hp1', second is 'hp2', etc.
&nbsp;   
{'hp1': {  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 1': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 2': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 3': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ...  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }  
 'hp2': {  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 'Chapter 1': (chapter title, chapter text),  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; ...  
         &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; }  
 &nbsp;...  
}  

In [50]:
for book in hp:
    print(book)

Harry Potter and the Sorcerer's Stone
Harry Potter and the Chamber of Secrets
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Goblet of Fire
Harry Potter and the Order of the Phoenix
Harry Potter and the Half-Blood Prince
Harry Potter and the Deathly Hallows


In [52]:
language = "english"
sentence_count = 5
 
parser = PlaintextParser(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1], Tokenizer(language))

In [53]:
from sumy.summarizers.lex_rank import LexRankSummarizer 
summarizer = LexRankSummarizer(Stemmer(language))
summarizer.stop_words = get_stop_words(language)
#Summarize the document with 2 sentences
summary = summarizer(parser.document, sentence_count) 
for sentence in summary:
    print(sentence)

"The Potters, that's right, that's what I heard —" "— yes, their son, Harry —" Mr. Dursley stopped dead.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
Dumbledore turned and walked back down the street.


In [54]:
from sumy.summarizers.luhn import LuhnSummarizer
summarizer_1 = LuhnSummarizer(Stemmer(language))
summarizer_1.stop_words = get_stop_words(language)
summary_1 = summarizer_1(parser.document, sentence_count)
for sentence in summary_1:
    print(sentence)

It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
He didn't see the owls swoop- ing past in broad daylight, though people down in the street did; they pointed and gazed open-mouthed as owl after owl sped overhead.
No one knows why, or how, but they're saying that when he couldn't kill Harry Potter, Voldemort's power somehow broke — and that's why he's gone."
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
G'night, Professor McGonagall — Professor Dumbledore, sir."


In [55]:
from sumy.summarizers.lsa import LsaSummarizer
summarizer_2 = LsaSummarizer(Stemmer(language))
summarizer_2.stop_words = get_stop_words(language)
summary_2 = summarizer_2(parser.document, sentence_count)
for sentence in summary_2:
    print(sentence)

He dashed back across the road, hurried up to his office, snapped at his secretary not to disturb him, seized his telephone, and had almost finished dialing his home number when he changed his mind.
It seemed that Professor McGonagall had reached the point she was most anxious to discuss, the real reason she had been waiting on a cold, hard wall all day, for neither as a cat nor as a woman had she fixed Dumbledore with such a piercing stare as she did now.
He looked simply too big to be allowed, and so wild — long tangles of bushy black hair and beard hid most of his face, he had hands the size of trash can lids, and his feet in their leather boots were like baby dolphins.
For a full minute the three of them stood and looked at the little bundle; Hagrid's shoulders shook, Professor McGonagall blinked furiously, and the twinkling light that usually shone from Dumbledore's eyes seemed to have gone out.
A breeze ruffled the neat hedges of Privet Drive, which lay silent and tidy under the 

In [56]:
from sumy.summarizers.text_rank import TextRankSummarizer
summarizer_3 = TextRankSummarizer(Stemmer(language))
summarizer_3.stop_words = get_stop_words(language)
summary_3 = summarizer_3(parser.document, sentence_count)
for sentence in summary_3:
    print(sentence)

Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills.
He was a big, beefy man with hardly any neck, although he did have a very large mustache.
Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.


/anaconda3/lib/python3.6/site-packages/sumy/summarizers/text_rank.py:68: RuntimeWarning: invalid value encountered in true_divide
  weights /= weights.sum(axis=1)[:, numpy.newaxis]


In [57]:
from sumy.summarizers.edmundson import EdmundsonSummarizer
summarizer_4 = EdmundsonSummarizer(Stemmer(language))
summarizer_4.stop_words = get_stop_words(language)
summarizer_4.bonus_words = hp['hp1']['Chapter 1'][0].split()
summarizer_4.stigma_words = ['zdfgthdvndadv']
summarizer_4.null_words = ['zdfgthdvndadv']
summary_4 = summarizer_4(parser.document, sentence_count)
for sentence in summary_4:
    print(sentence)

KeyError: 'hp1'

In [ ]:
from sumy.summarizers.edmundson import EdmundsonSummarizer
summarizer_4 = EdmundsonSummarizer(Stemmer(language))
summarizer_4.stop_words = get_stop_words(language)
summarizer_4.bonus_words = ['zdfgthdvndadv']#hp['hp1']['Chapter 1'][0].split()
summarizer_4.stigma_words = ['zdfgthdvndadv']
summarizer_4.null_words = ['zdfgthdvndadv']
summary_4 = summarizer_4(parser.document, sentence_count)
for sentence in summary_4:
    print(sentence)

In [ ]:
from sumy.summarizers.sum_basic import SumBasicSummarizer
summarizer_5 = SumBasicSummarizer(Stemmer(language))
summarizer_5.stop_words = get_stop_words(language)
summary_5 = summarizer_5(parser.document, 5)
for sentence in summary_5:
    print(sentence)

In [ ]:
from sumy.summarizers.kl import KLSummarizer
summarizer_6 = KLSummarizer(Stemmer(language))
summarizer_6.stop_words = get_stop_words(language)
summary_6 = summarizer_6(parser.document, sentence_count)
for sentence in summary_6:
    print(sentence)

In [ ]:
from sumy.summarizers.reduction import ReductionSummarizer
summarizer_7 = ReductionSummarizer(Stemmer(language))
summarizer_7.stop_words = get_stop_words(language)
summary_7 = summarizer_7(parser.document, sentence_count)
for sentence in summary_7:
    print(sentence)

In [58]:
from sumy.summarizers.lex_rank import LexRankSummarizer 
from sumy.summarizers.luhn import LuhnSummarizer
from sumy.summarizers.lsa import LsaSummarizer
from sumy.summarizers.text_rank import TextRankSummarizer
from sumy.summarizers.edmundson import EdmundsonSummarizer
from sumy.summarizers.sum_basic import SumBasicSummarizer
from sumy.summarizers.kl import KLSummarizer
from sumy.summarizers.reduction import ReductionSummarizer

In [61]:
def summarize(text, summarizer, sentence_count, language='english'):
    summarizer = summarizer(Stemmer(language))
    summarizer.stop_words = get_stop_words(language)
    if isinstance(summarizer, EdmundsonSummarizer):
        summarizer.bonus_words = hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][0].split()
        summarizer.stigma_words = ('and', 'the')
        summarizer.null_words = ('and', 'the')
    summary = summarizer(PlaintextParser(text, Tokenizer(language)).document, sentence_count)
    return summary

In [60]:
def print_summary(summary):
    for sentence in summary:
        print(sentence)

In [62]:
for summarizer in [LexRankSummarizer, LuhnSummarizer, LsaSummarizer, TextRankSummarizer,
                   EdmundsonSummarizer, SumBasicSummarizer, KLSummarizer, ReductionSummarizer]:
    print('----' + summarizer.__name__ + '----')
    print_summary(summarize(hp["Harry Potter and the Sorcerer's Stone"]['Chapter 1'][1], summarizer, sentence_count))
    print()

----LexRankSummarizer----
"The Potters, that's right, that's what I heard —" "— yes, their son, Harry —" Mr. Dursley stopped dead.
Twelve times he clicked the Put-Outer, until the only lights left on the whole street were two tiny pinpricks in the distance, which were the eyes of the cat watching him.
Dumbledore slipped the Put-Outer back inside his cloak and set off down the street toward number four, where he sat down on the wall next to the cat.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.
Dumbledore turned and walked back down the street.

----LuhnSummarizer----
It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
He didn't see the owls swoop-

/anaconda3/lib/python3.6/site-packages/sumy/summarizers/text_rank.py:68: RuntimeWarning: invalid value encountered in true_divide
  weights /= weights.sum(axis=1)[:, numpy.newaxis]


Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much.
They were the last people you'd expect to be involved in anything strange or mysterious, because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made drills.
He was a big, beefy man with hardly any neck, although he did have a very large mustache.
Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors.

----EdmundsonSummarizer----
This boy was another good reason for keeping the Potters away; they didn't want Dudley mixing with a child like that.
Mr. Dursley couldn't bear people who dressed in funny clothes — the getups you saw on young people!
He was sure there were lots of people called Potter who had a son called Harry.
"You don't mean — you can't mean the people who live

In [63]:
for book in hp:
    print(book)
    for chapter in hp[book]:
        print()
        print('{} - {}'.format(chapter, hp[book][chapter][0]))
        print_summary(summarize(hp[book][chapter][1], LuhnSummarizer, 2))
    print()
    print()

Harry Potter and the Sorcerer's Stone

Chapter 1 - THE BOY WHO LIVED
It was now reading the sign that said Privet Drive — no, looking at the sign; cats couldn't read maps or signs.
"But I c-c-can't stand it — Lily an' James dead — an' poor little Harry off ter live with Muggles —" "Yes, yes, it's all very sad, but get a grip on yourself, Hagrid, or we'll be found," Professor McGonagall whispered, patting Hagrid gingerly on the arm as Dumbledore stepped over the low garden wall and walked to the front door.

Chapter 2 - THE VANISHING GLASS
He had a large pink face, not much neck, small, watery blue eyes, and thick blond hair that lay smoothly on his thick, fat head.
Everybody knew that Dudley's gang hated that odd Harry Potter in his baggy old clothes and broken glasses, and nobody liked to disagree with Dudley's gang.

Chapter 3 - THE LETTERS FROM NO ONE
The Dursleys' house had four bedrooms: one for Uncle Vernon and Aunt Petunia, one for visitors (usually Uncle Vernon's sister, Marge)

Nose still stinging where it had hit the hearth, Harry made his way swiftly and silently toward the door, but before he'd got halfway toward it, two people appeared on the other side of the glass — and one of them was the  very last person Harry wanted to meet when he was lost, covered in soot, and wearing broken glasses: Draco Malfoy.
Harry looked quickly around and spotted a large black cabinet to his left; he shot inside it and pulled the doors closed, leaving a small crack to peer through.

Chapter 5 - THE WHOMPING WILLOW
When at last they were all in the car, Mrs. Weasley glanced into the back seat, where Harry, Ron, Fred, George, and Percy were all sitting comfortably side by side, and said, "Muggles do know more than we give them credit for, don't they?"
Both trolleys hit the barrier and bounced backward; Ron's trunk fell off with a loud thump, Harry was knocked off his feet, and Hedwig's cage bounced onto the shiny floor, and she rolled away, shrieking indignantly; people all a

Ron called Professor McGonagall a lot of names that greatly annoyed Hermione; Hermione assumed an "all-for-the-best" expression that made Ron even angrier, and Harry had to endure everyone in the class talking loudly and happily about what they were going to do first, once they got into Hogsmeade.
He accompanied them to the entrance hall, where Filch, the caretaker, was standing inside the front doors, checking off names against a long list, peering suspiciously into every face, and making sure that no one was sneaking out who shouldn't be going.

Chapter 9 - GRIM DEFEAT
Then, at their final training session before Saturday's match, Oliver Wood gave his team some unwelcome news.
The third time this happened, Wood talked for so long that Harry suddenly realized he was ten minutes late for Defense Against the Dark Arts, and set off at a run with Wood shouting after him, "Diggory's got a very fast swerve, Harry, so you might want to try looping him —" Harry skidded to a halt outside the D

"I'll send Bill, Charlie, and Percy along around midday," Mrs. Weasley said to Mr. Weasley, as he, Harry, Ron, Hermione, and Ginny set off across the dark yard after Fred and George.
Harry, having been thinking about thousands of wizards speeding toward the Quidditch World Cup, sped up to walk with Mr. Weasley.

Chapter 7 - BAGMAN AND CROUCH
"Let's go and have a look," said Harry, pointing to a large patch of tents upheld, where the Bulgarian flag — white, green, and red — was fluttering in the breeze.
Though Ron purchased a dancing shamrock hat and a large green rosette, he also bought a small figure of Viktor Krum, the Bulgarian Seeker.

Chapter 8 - THE QUIDDITCH WORLD CUP
Harry spun the speed dial on his Omnioculars back to normal, watching closely as Mostafa mounted his broomstick and kicked the crate open — four balls burst into the air: the scarlet Quaffle, the two black Bludgers, and (Harry saw it for the briefest moment, before it sped out of sight) the minuscule, winged Golden

Pansy's pug-like face peered excitedly around Goyle's broad back as Harry, Ron, and Hermione approached.
The food Sirius had told them to bring was inCHAPTER TWENTY-SEVEN Harry's bag; they had sneaked a dozen chicken legs, a loaf of bread, and a flask of pumpkin juice from the lunch table.

Chapter 28 - THE MADNESS OF MR. CROUCH
"He is nosing — hic — into my master's — hic — private and secret — hic — Winky is a good house-elf — hic — Winky keeps her silence — hic — people trying to — hic — pry and poke — hic —" Winky's eyelids drooped and suddenly, without warning, she slid off her stool into the hearth, snoring loudly.
The bell rang across the grounds for lunch; the rest of the class set off back to the castle, but Harry, Ron, and Hermione stayed behind to help Hagrid put the nifflers back in their boxes.

Chapter 29 - THE DREAM
All three of them were so tired they could happily have put their heads down on the desks and slept; even Hermione wasn't taking her usual notes, but was sit

...

They looked at one another for a moment, then Hermione pulled the newspaper back toward her, closed it, glared for a moment at the pictures of the ten escaped Death Eaters on the front, then leapt to her feet.
For new signs had appeared on the house notice boards the morning after news of the Azkaban breakout: —  by order of — The High Inquisitor of Hogwarts Teachers are hereby banned from giving students any information that is not strictly related to the subjects they are paid to teach.

Chapter 25 - SEEN AND UNFORESEEN
"CHAPTER TWENTY-SIX Harry was a hero in the Gryffindor common room that night; daringly, Fred and George had put an Enlargement Charm on the front cover of The Quibbler and hung it on the wall, so that Harry's giant head gazed down upon the proceedings, occasionally saying things like "The Ministry are morons" and "Eat dung, Umbridge" in a booming voice.
He was hurtling along the corridor toward the Department of Mysteries, past the blank stone walls, past the torches

Harry was too busy digesting the horrible idea of Inferi to have much attention left for anything else, but as they reached the front gate, Dumbledore stopped dead and Harry walked into him.
Slughorn's watery eyes slid over Harry's scar, this time taking in the rest of his face.

Chapter 5 - AN EXCESS OF PHLEGM
But when Harry arrived downstairs ten minutes later, fully dressed and carrying his empty breakfast tray, it was to find Hermione sitting at the kitchen table in great agitation, while Mrs. Weasley tried to lessen her resemblance to half a panda.
Ordinary Wizarding Level Results Pass Grades  Fail Grades Outstanding (O)  Poor (P) Exceeds Expectations (E)  Dreadful (D) Acceptable (A)  Troll (T) Harry James Potter has achieved: Astronomy A Care of Magical Creatures  E Charms  E Defense Against the Dark Arts  O Divination  P Herbology  E History of Magic  D Potions  E Transfiguration  E Harry read the parchment through several times, his breathing becoming easier with each reading.


There was a loud bang and the bin behind Harry exploded; Harry attempted a Leg-Locker Curse that backfired off the wall behind Malfoy's ear and smashed the cistern beneath Moaning Myrtle, who screamed loudly; water poured everywhere and Harry slipped as Malfoy, his face contorted, cried, "Cruci —" "SECTUMSEMPRA!"
Harry stuffed a bent card into the box at random and hurried out of the door before Snape could change his mind, racing back up the stone steps, straining his ears to hear a sound from the pitch, but all was quiet.

Chapter 25 - THE SEER OVERHEARD
"You'd think people had better things to gossip about," said Ginny, as she sat on the common room floor, leaning against Harry's legs and reading the Daily Prophet.
"Professor Trelawney was just in the Room of Requirement, trying to hide her sherry bottles, and she heard Malfoy whooping, celebrating!

Chapter 26 - THE CAVE
But though gashes appeared in their sodden rags and their icy skin, they had no blood to spill: They walked on, 

His long black hair and beard were streaked with silver, and a great overhanging forehead shadowed his glinting eyes, putting Harry in mind of a crab looking out from beneath a rock.
1 will contact (has stayed with Weasley family previously) "Undesirable Number One," Harry muttered under his breath as he replaced Mr. Weasley's folder and shut the drawer.

Chapter 14 - THE THIEF
"If we're staying, we should put some protective enchantments around the place," she replied, and raising her wand, she began to walk in a wide circle around Harry and Ron, murmuring incantations as she went.
He looked up at Hermione, whose bushy hair obscured the tiny patch of sky visible through the dark branches high above them.

Chapter 15 - THE GOBLIN'S REVENGE
As they lifted the empty portrait of Phineas Nigellus free of Hermione's bag, she kept her wand pointing at it, ready to cast a spell at any moment.
For one bewildered moment he thought that Ron had left the tent, then realized that Ron was lying in 

Harry remained kneeling at Snape's side, simply staring down at him, until quite suddenly a high, cold voice spoke so close to them that Harry jumped to his feet, the flask gripped tightly in his hands, thinking that Voldemort had reentered the room.
They were back in Dumbledore's office, the windows dark, and Fawkes sat silent as Snape sat quite still, as Dumbledore walked around him, talking.

Chapter 34 - THE FOREST AGAIN
The images of Fred, Lupin, and Tonks lying dead in the Great Hall forced their way back into his mind's eye, and for a moment he could hardly breathe: Death was impatient.
Two figures emerged from behind a nearby tree: Their wands flared, and Harry saw Yaxley and Dolohov peering into the darkness, directly at the place Harry, his mother and father and Sirius and Lupin stood.

Chapter 35 - KING'S CROSS
Stunned, Harry followed as Dumbledore strode away from where the flayed child lay whimpering, leading him to two seats that Harry had not previously noticed, set some